In [27]:
import mlflow
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing
import tensorflow_addons as tfa

from tensorflow.keras.callbacks import EarlyStopping

from tabtransformertf.models.fttransformer import FTTransformerEncoder, FTTransformer
from tabtransformertf.utils.preprocessing import df_to_dataset

import catboost as cb
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MeanSquaredError

In [3]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams.update({'font.size': 15})

In [9]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("income")

2022/12/21 19:50:42 INFO mlflow.tracking.fluent: Experiment with name 'income' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/1', creation_time=1671652242067, experiment_id='1', last_update_time=1671652242067, lifecycle_stage='active', name='income', tags={}>

## Load Data

In [4]:
dset = fetch_california_housing()
data = dset['data']
y = dset['target']
LABEL = dset['target_names'][0]

NUMERIC_FEATURES = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Longitude', 'Latitude']
FEATURES = NUMERIC_FEATURES

data = pd.DataFrame(data, columns=dset['feature_names'])
data[LABEL] = y

data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [5]:
train_data, test_data = train_test_split(data, test_size=0.2)
print(f"Train dataset shape: {train_data.shape}")
print(f"Test dataset shape: {test_data.shape}")

Train dataset shape: (16512, 9)
Test dataset shape: (4128, 9)


## Data Processing

In [6]:
X_train, X_val = train_test_split(train_data, test_size=0.2)

sc = StandardScaler()
X_train.loc[:, NUMERIC_FEATURES] = sc.fit_transform(X_train[NUMERIC_FEATURES])
X_val.loc[:, NUMERIC_FEATURES] = sc.transform(X_val[NUMERIC_FEATURES])
test_data.loc[:, NUMERIC_FEATURES] = sc.transform(test_data[NUMERIC_FEATURES])

## Baseline

In [10]:
mlflow.sklearn.autolog(disable=True)

with mlflow.start_run(run_name='rf_baseline'):
    params = {
        "n_estimators": 100,
        "max_depth": 20
    }

    mlflow.set_tag("model_name", "RF")
    mlflow.log_params(params)

    rf = RandomForestRegressor(n_estimators=100, max_depth=20)
    rf.fit(X_train[FEATURES], X_train[LABEL])

    rf_preds = rf.predict(test_data[FEATURES])
    rf_rms = mean_squared_error(test_data[LABEL], rf_preds, squared=False)

    mlflow.log_metric("test_rmse", rf_rms)
    mlflow.sklearn.log_model(rf, "sk_models")

## CatBoost

In [11]:
catb_train_dataset = cb.Pool(X_train[FEATURES], X_train[LABEL]) 
catb_val_dataset = cb.Pool(X_val[FEATURES], X_val[LABEL]) 
catb_test_dataset = cb.Pool(test_data[FEATURES], test_data[LABEL])

In [12]:
with mlflow.start_run(run_name="catboost"):
    mlflow.set_tag("model_name", "CatBoost")
    catb = cb.CatBoostRegressor()
    catb.fit(catb_train_dataset, eval_set=catb_val_dataset, early_stopping_rounds=50)
    catb_preds = catb.predict(catb_test_dataset)
    catb_rms = mean_squared_error(test_data[LABEL], catb_preds, squared=False)

    mlflow.log_metric("test_rmse", catb_rms)
    mlflow.catboost.log_model(catb, "cb_models")

Learning rate set to 0.076361
0:	learn: 1.1040538	test: 1.1231963	best: 1.1231963 (0)	total: 64.8ms	remaining: 1m 4s
1:	learn: 1.0613534	test: 1.0796379	best: 1.0796379 (1)	total: 66.9ms	remaining: 33.4s
2:	learn: 1.0225802	test: 1.0404682	best: 1.0404682 (2)	total: 68.9ms	remaining: 22.9s
3:	learn: 0.9863285	test: 1.0031969	best: 1.0031969 (3)	total: 71.1ms	remaining: 17.7s
4:	learn: 0.9536253	test: 0.9714099	best: 0.9714099 (4)	total: 72.9ms	remaining: 14.5s
5:	learn: 0.9222412	test: 0.9396970	best: 0.9396970 (5)	total: 74.9ms	remaining: 12.4s
6:	learn: 0.8956087	test: 0.9122648	best: 0.9122648 (6)	total: 77ms	remaining: 10.9s
7:	learn: 0.8702443	test: 0.8866264	best: 0.8866264 (7)	total: 79ms	remaining: 9.79s
8:	learn: 0.8470759	test: 0.8636610	best: 0.8636610 (8)	total: 80.7ms	remaining: 8.89s
9:	learn: 0.8264564	test: 0.8424327	best: 0.8424327 (9)	total: 82.5ms	remaining: 8.16s
10:	learn: 0.8081711	test: 0.8241009	best: 0.8241009 (10)	total: 84.2ms	remaining: 7.57s
11:	learn: 0.79

## MLP

In [114]:
def build_mlp(params):
    mlp = Sequential([
        Dense(params["layer1_size"], activation=params['activation']),
        Dropout(params['dropout_rate']),
        Dense(params["layer2_size"], activation=params['activation']),
        Dropout(params['dropout_rate']),
        Dense(params["layer3_size"], activation=params['activation']),
        Dense(1, activation='relu')
    ])
    return mlp

def train_mlp(mlp, train_params, train_dataset, val_dataset):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=train_params["learning_rate"],
        weight_decay=train_params["weight_decay"],
    )
    mlp.compile(
        optimizer=optimizer,
        loss=MeanSquaredError(name="mse"),
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")]
    )

    early = EarlyStopping(
        monitor="val_loss",
        mode="min",
        patience=train_params["early_stop_patience"],
        restore_best_weights=True,
    )
    callback_list = [early]

    hist = mlp.fit(
        train_dataset,
        epochs=train_params["num_epochs"],
        validation_data=val_dataset,
        callbacks=callback_list,
    )
    return mlp


def mlp_mlflow_run(
    name,
    mlp_params,
    train_params,
    train_dataset,
    val_dataset,
    test_dataset,
    y_test,
):
    with mlflow.start_run(run_name=name):
        mlflow.log_params(mlp_params)
        mlflow.log_params(train_params)
        mlflow.set_tag("model_name", "MLP")
        mlp = build_mlp(mlp_params)
        mlp = train_mlp(mlp, train_params, train_dataset, val_dataset)
        test_preds = mlp.predict(test_dataset)
        test_rms = mean_squared_error(
            y_test, test_preds.ravel(), squared=False
        )
        mlflow.log_metric("test_rmse", test_rms)
        mlflow.tensorflow.log_model(mlp, "tf_models")

In [119]:
# To TF Dataset
mlp_train_ds = tf.data.Dataset.from_tensor_slices((X_train[FEATURES], X_train[LABEL])).batch(512).shuffle(512*4).prefetch(512)
mlp_val_ds = tf.data.Dataset.from_tensor_slices((X_val[FEATURES], X_val[LABEL])).batch(512).shuffle(512*4).prefetch(512)
mlp_test_ds = tf.data.Dataset.from_tensor_slices(test_data[FEATURES]).batch(512).prefetch(512)

mlp_params = {
    "layer1_size": 512,
    "layer2_size": 128,
    "layer3_size": 64,
    "dropout_rate": 0.3,
    "activation": 'relu'

}
train_params = dict(
    learning_rate=0.001, weight_decay=0.00001, early_stop_patience=10, num_epochs=1000
)

mlp_mlflow_run(
    "mlp_base",
    mlp_params,
    train_params,
    mlp_train_ds,
    mlp_val_ds,
    mlp_test_ds,
    test_data[LABEL],
)

Epoch 1/1000
26/26 [==============================] - 2s 13ms/step - loss: 2.3211 - rmse: 1.5235 - val_loss: 0.8842 - val_rmse: 0.9403
Epoch 2/1000
26/26 [==============================] - 0s 5ms/step - loss: 0.9310 - rmse: 0.9649 - val_loss: 0.6568 - val_rmse: 0.8104
Epoch 3/1000
26/26 [==============================] - 0s 7ms/step - loss: 0.6655 - rmse: 0.8158 - val_loss: 0.5097 - val_rmse: 0.7140
Epoch 4/1000
26/26 [==============================] - 0s 5ms/step - loss: 0.5503 - rmse: 0.7418 - val_loss: 0.4466 - val_rmse: 0.6683
Epoch 5/1000
26/26 [==============================] - 0s 5ms/step - loss: 0.4875 - rmse: 0.6982 - val_loss: 0.4210 - val_rmse: 0.6489
Epoch 6/1000
26/26 [==============================] - 0s 6ms/step - loss: 0.4632 - rmse: 0.6806 - val_loss: 0.3972 - val_rmse: 0.6302
Epoch 7/1000
26/26 [==============================] - 0s 8ms/step - loss: 0.4509 - rmse: 0.6715 - val_loss: 0.3833 - val_rmse: 0.6191
Epoch 8/1000
26/26 [==============================] - 0s 9ms/

INFO:tensorflow:Assets written to: /var/folders/66/1klxbkpn5vdgpvqwt_hmtn5c0000gn/T/tmpm_z7_2xf/model/data/model/assets
2022/12/22 14:21:01 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/66/1klxbkpn5vdgpvqwt_hmtn5c0000gn/T/tmpm_z7_2xf/model, flavor: tensorflow), fall back to return ['tensorflow==2.7.0']. Set logging level to DEBUG to see the full traceback.


In [ ]:
mlp_mlflow_run

## FT Transformers

In [78]:
# To TF Dataset
train_dataset = df_to_dataset(X_train[FEATURES + [LABEL]], LABEL, shuffle=True)
val_dataset = df_to_dataset(X_val[FEATURES + [LABEL]], LABEL, shuffle=False)  # No shuffle
test_dataset = df_to_dataset(test_data[FEATURES], shuffle=False) # No target, no shuffle

/Users/antonsruberts/personal/TabTransformerTF/tabtransformertf/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]
/Users/antonsruberts/personal/TabTransformerTF/tabtransformertf/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]
/Users/antonsruberts/personal/TabTransformerTF/tabtransformertf/utils/preprocessing.py:27: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]


In [97]:
def build_fttransformer(
    params_to_log, params_to_skip, out_dim=1, out_activation="relu"
):
    # Define encoder
    ft_encoder = FTTransformerEncoder(
        **params_to_log,
        **params_to_skip
    )
    # Add prediction head to the encoder
    ft_transformer = FTTransformer(
        encoder=ft_encoder,
        out_dim=out_dim,
        out_activation=out_activation,
    )

    return ft_transformer


def train_model(model, train_params, train_dataset, val_dataset):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=train_params["learning_rate"],
        weight_decay=train_params["weight_decay"],
    )

    model.compile(
        optimizer=optimizer,
        loss={
            "output": tf.keras.losses.MeanSquaredError(name="mse"),
            "importances": None,
        },
        metrics={
            "output": [tf.keras.metrics.RootMeanSquaredError(name="rmse")],
            "importances": None,
        },
    )

    early = EarlyStopping(
        monitor="val_output_loss",
        mode="min",
        patience=train_params["early_stop_patience"],
        restore_best_weights=True,
    )
    callback_list = [early]

    hist = model.fit(
        train_dataset,
        epochs=train_params["num_epochs"],
        validation_data=val_dataset,
        callbacks=callback_list,
    )
    return model


In [103]:
mlflow.tensorflow.autolog(disable=True)


def fttransformer_mlflow_run(
    name,
    encoder_params,
    train_params,
    params_to_skip,
    train_dataset,
    val_dataset,
    test_dataset,
    y_test,
):
    with mlflow.start_run(run_name=name):
        mlflow.set_tag("model_name", "FTTransformer")
        # Log the params
        mlflow.log_params(encoder_params)
        mlflow.log_params(train_params)
        # Build and train
        ft_transformer = build_fttransformer(
            encoder_params,
            params_to_skip,
            out_dim=1,
            out_activation="relu",
        )
        ft_transformer = train_model(
            ft_transformer, train_params, train_dataset, val_dataset
        )
        # Evaluate
        test_preds = ft_transformer.predict(test_dataset)
        test_rms = mean_squared_error(
            y_test, test_preds["output"].ravel(), squared=False
        )
        mlflow.log_metric("test_rmse", test_rms)
        mlflow.tensorflow.log_model(ft_transformer, "tf_models")


In [104]:
train_params = dict(
    learning_rate=0.001, weight_decay=0.00001, early_stop_patience=10, num_epochs=1000
)

params_to_skip = dict(
    numerical_data=X_train[NUMERIC_FEATURES].values,
    categorical_data=None,
    y=X_train[LABEL].values
)

### Linear Embeddings

In [105]:
linear_embeddings_params = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type="linear",
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

fttransformer_mlflow_run(
    name='linear',
    encoder_params=linear_embeddings_params,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000
26/26 [==============================] - 9s 242ms/step - loss: 1.0839 - output_loss: 1.0839 - output_rmse: 1.0411 - val_loss: 0.7125 - val_output_loss: 0.7125 - val_output_rmse: 0.8441
Epoch 2/1000
26/26 [==============================] - 7s 253ms/step - loss: 0.6225 - output_loss: 0.6225 - output_rmse: 0.7890 - val_loss: 0.4414 - val_output_loss: 0.4414 - val_output_rmse: 0.6644
Epoch 3/1000
26/26 [==============================] - 6s 231ms/step - loss: 0.4873 - output_loss: 0.4873 - output_rmse: 0.6981 - val_loss: 0.4384 - val_output_loss: 0.4384 - val_output_rmse: 0.6621
Epoch 4/1000
26/26 [==============================] - 6s 221ms/step - loss: 0.4539 - output_loss: 0.4539 - output_rmse: 0.6737 - val_loss: 0.4481 - val_output_loss: 0.4481 - val_output_rmse: 0.6694
Epoch 5/1000
26/26 [==============================] - 6s 218ms/step - loss: 0.4194 - output_loss: 0.4194 - output_rmse: 0.6476 - val_loss: 0.3783 - val_output_loss: 0.3783 - val_output_rmse: 0.6151
Epoch 6/10

INFO:tensorflow:Assets written to: /var/folders/66/1klxbkpn5vdgpvqwt_hmtn5c0000gn/T/tmpl1a6lq8v/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/66/1klxbkpn5vdgpvqwt_hmtn5c0000gn/T/tmpl1a6lq8v/model/data/model/assets
/Users/antonsruberts/miniconda/envs/blog/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
2022/12/21 23:19:27 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/66/1klxbkpn5vdgpvqwt_hmtn5c0000gn/T/tmpl1a6lq8v/model, flavor: tensorflow), fall back to return ['tensorflow==2.7.0']. Set logging level to DEBUG to see the full traceback.


### Periodic

In [106]:
periodic_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='periodic',
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

fttransformer_mlflow_run(
    name='periodic',
    encoder_params=periodic_params_to_log,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000
26/26 [==============================] - 11s 288ms/step - loss: 1.2616 - output_loss: 1.2616 - output_rmse: 1.1232 - val_loss: 0.6716 - val_output_loss: 0.6716 - val_output_rmse: 0.8195
Epoch 2/1000
26/26 [==============================] - 8s 285ms/step - loss: 0.5666 - output_loss: 0.5666 - output_rmse: 0.7527 - val_loss: 0.4696 - val_output_loss: 0.4696 - val_output_rmse: 0.6853
Epoch 3/1000
26/26 [==============================] - 6s 242ms/step - loss: 0.4252 - output_loss: 0.4252 - output_rmse: 0.6521 - val_loss: 0.3980 - val_output_loss: 0.3980 - val_output_rmse: 0.6309
Epoch 4/1000
26/26 [==============================] - 6s 235ms/step - loss: 0.3822 - output_loss: 0.3822 - output_rmse: 0.6182 - val_loss: 0.3677 - val_output_loss: 0.3677 - val_output_rmse: 0.6064
Epoch 5/1000
26/26 [==============================] - 6s 236ms/step - loss: 0.3582 - output_loss: 0.3582 - output_rmse: 0.5985 - val_loss: 0.3496 - val_output_loss: 0.3496 - val_output_rmse: 0.5913
Epoch 6/1

INFO:tensorflow:Assets written to: /var/folders/66/1klxbkpn5vdgpvqwt_hmtn5c0000gn/T/tmpngoprskv/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/66/1klxbkpn5vdgpvqwt_hmtn5c0000gn/T/tmpngoprskv/model/data/model/assets
/Users/antonsruberts/miniconda/envs/blog/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
2022/12/21 23:24:18 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/66/1klxbkpn5vdgpvqwt_hmtn5c0000gn/T/tmpngoprskv/model, flavor: tensorflow), fall back to return ['tensorflow==2.7.0']. Set logging level to DEBUG to see the full traceback.


### PLE - Quantile Binning

In [107]:
pleq_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='ple',
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

pleq_params_to_skip = params_to_skip.copy()
pleq_params_to_skip['y'] = None

fttransformer_mlflow_run(
    name='ple_quantile',
    encoder_params=pleq_params_to_log,
    train_params=train_params,
    params_to_skip=pleq_params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000
26/26 [==============================] - 148s 2s/step - loss: 1.3924 - output_loss: 1.3924 - output_rmse: 1.1800 - val_loss: 0.6142 - val_output_loss: 0.6142 - val_output_rmse: 0.7837
Epoch 2/1000
26/26 [==============================] - 7s 274ms/step - loss: 0.5275 - output_loss: 0.5275 - output_rmse: 0.7263 - val_loss: 0.4237 - val_output_loss: 0.4237 - val_output_rmse: 0.6509
Epoch 3/1000
26/26 [==============================] - 7s 266ms/step - loss: 0.4062 - output_loss: 0.4062 - output_rmse: 0.6373 - val_loss: 0.3761 - val_output_loss: 0.3761 - val_output_rmse: 0.6133
Epoch 4/1000
26/26 [==============================] - 7s 265ms/step - loss: 0.3623 - output_loss: 0.3623 - output_rmse: 0.6019 - val_loss: 0.3621 - val_output_loss: 0.3621 - val_output_rmse: 0.6018
Epoch 5/1000
26/26 [==============================] - 7s 256ms/step - loss: 0.3395 - output_loss: 0.3395 - output_rmse: 0.5826 - val_loss: 0.3201 - val_output_loss: 0.3201 - val_output_rmse: 0.5658
Epoch 6/100

INFO:tensorflow:Assets written to: /var/folders/66/1klxbkpn5vdgpvqwt_hmtn5c0000gn/T/tmp9tk7cqp0/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/66/1klxbkpn5vdgpvqwt_hmtn5c0000gn/T/tmp9tk7cqp0/model/data/model/assets
/Users/antonsruberts/miniconda/envs/blog/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
2022/12/21 23:42:47 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/66/1klxbkpn5vdgpvqwt_hmtn5c0000gn/T/tmp9tk7cqp0/model, flavor: tensorflow), fall back to return ['tensorflow==2.7.0']. Set logging level to DEBUG to see the full traceback.


### PLE - Target Binning

In [109]:
plet_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='ple',
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
    task='regression',
    ple_tree_params = {
        "min_samples_leaf": 20,
    }
)


fttransformer_mlflow_run(
    name='ple_target',
    encoder_params=plet_params_to_log,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000
26/26 [==============================] - 357s 3s/step - loss: 0.9989 - output_loss: 0.9989 - output_rmse: 0.9995 - val_loss: 0.4648 - val_output_loss: 0.4648 - val_output_rmse: 0.6818
Epoch 2/1000
26/26 [==============================] - 7s 280ms/step - loss: 0.4568 - output_loss: 0.4568 - output_rmse: 0.6758 - val_loss: 0.3808 - val_output_loss: 0.3808 - val_output_rmse: 0.6171
Epoch 3/1000
26/26 [==============================] - 7s 286ms/step - loss: 0.3834 - output_loss: 0.3834 - output_rmse: 0.6192 - val_loss: 0.3446 - val_output_loss: 0.3446 - val_output_rmse: 0.5870
Epoch 4/1000
26/26 [==============================] - 8s 287ms/step - loss: 0.3463 - output_loss: 0.3463 - output_rmse: 0.5885 - val_loss: 0.3175 - val_output_loss: 0.3175 - val_output_rmse: 0.5635
Epoch 5/1000
26/26 [==============================] - 7s 283ms/step - loss: 0.3306 - output_loss: 0.3306 - output_rmse: 0.5750 - val_loss: 0.2972 - val_output_loss: 0.2972 - val_output_rmse: 0.5452
Epoch 6/100

INFO:tensorflow:Assets written to: /var/folders/66/1klxbkpn5vdgpvqwt_hmtn5c0000gn/T/tmp00brey19/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/66/1klxbkpn5vdgpvqwt_hmtn5c0000gn/T/tmp00brey19/model/data/model/assets
/Users/antonsruberts/miniconda/envs/blog/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
2022/12/22 00:10:05 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/66/1klxbkpn5vdgpvqwt_hmtn5c0000gn/T/tmp00brey19/model, flavor: tensorflow), fall back to return ['tensorflow==2.7.0']. Set logging level to DEBUG to see the full traceback.


In [87]:
model_id = "" # take it from mlflow
loaded_ft = mlflow.tensorflow.load_model(model_id)